In [1]:
from Instrument import Instrument
import string
import random
import Types
from time import sleep

# This notebook is used for testing ZND-RC functionality

## Connection

In [2]:
resource_string = 'TCPIP::192.168.0.169::hislip0'
instrument = Instrument(resource_string)
instrument.greet()


Hello, I am: 'Rohde-Schwarz,ZND-2Port,1328517092100889,2.80'
RsInstrument driver version: 1.9.0.52
Visa manufacturer: Rohde & Schwarz GmbH
Instrument full name: ZND-2Port
Instrument installed options: K1,K6


## Instrument Settings

In [3]:
instrument.reset()

In [4]:
instrument.set_visa_timeout = 3000

In [5]:
instrument.set_status_checking = True

## Correction

In [6]:
instrument.correction_load(1, 'N.cal')

In [7]:
instrument.correction_state()

"'CAI'"

## Traces

In [8]:
instrument.trace_create(1, 'MainTrace', 'S11')

In [9]:
instrument.trace_assign_to_window(1, 2, 'MainTrace')

In [10]:
instrument.trace_rename(2, 'MainTraceRenamed')

In [11]:
instrument.trace_set_points(1, 1000)

In [12]:
instrument.trace_copy('MemoryTrace', 'Trc1')

In [13]:
instrument.trace_assign_to_window(1, 3, 'MemoryTrace')

In [14]:
# generate save path of all traces
random_name = ''.join(random.choice(string.ascii_lowercase) for i in range(10)) 
save_path = r'C:\Users\Instrument\Desktop\test\{}.dat'.format(random_name)
# wait for a second 
# manual waiting like this should be avoided, instead, make sure trace is complete by initiating sweep
instrument.trace_save_all(1, save_path, 
                           formatted=True, 
                           save_format=Types.SaveFormat.COMPLEX, 
                           dec_separator=Types.DecimalSeparator.POINT, 
                           field_separator=Types.FieldSeparator.SEMICOLON)

## Sweep

In [ ]:
sweep_set_mode(1, Types.SweepMode.CONTINUOUS)

In [ ]:
sweep_set_mode_all(Types.SweepMode.SINGLE)

In [ ]:
sweep_initiate(1)

In [ ]:
sweep_initiate_all()

## Display

In [15]:
instrument.display_set_mode(Types.DisplayMode.OFF)

In [16]:
instrument.display_set_mode(Types.DisplayMode.ON)

## Touchscreen

In [17]:
instrument.touchscreen_set_lock(Types.TSLockMode.OFF)

In [18]:
instrument.window_create(2)

In [19]:
instrument.window_remove(2)

## Buttons

In [20]:
instrument.button_define(Types.ButtonNumber.ONE, "Say Hello", lambda: print("Hello World!"))
instrument.button_define(Types.ButtonNumber.TWO, "Say Bye", lambda: print("Bye World!"))

In [21]:
# pressing "Say Hello" button on the touchscreen will print "Hello World!" here
# pressing "Say Bye" button on the touchscreen will print "Bye World!" here
instrument.button_start_listening()

Hello World!
Bye World!


In [22]:
instrument.button_stop_listening()

## Transferring Files

In [1]:
# transfer_to_instrument(source, destination)

In [2]:
# transfer_from_instrument(source, destination)